In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-03-17 13:49:53.751077: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 13:49:53.929977: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-17 13:49:53.930020: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-17 13:49:55.182699: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
data = np.load(f'data/rectangle_1/04_training_data/X_norm.npy', allow_pickle=True)
data.shape

(17541, 100, 3)

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
latent_dim = 64

encoder_inputs = keras.Input(shape=(100, 3))
x = layers.Conv1D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv1D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 3)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 50, 32)       320         ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 25, 64)       6208        ['conv1d[0][0]']                 
                                                                                                  
 flatten (Flatten)              (None, 1600)         0           ['conv1d_1[0][0]']               
                                                                                            

2023-03-17 13:50:00.092327: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-17 13:50:00.092390: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-17 13:50:00.092427: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (srv-lab-t-251): /proc/driver/nvidia/version does not exist
2023-03-17 13:50:00.092697: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(25*64, activation="relu")(latent_inputs) # 25 = 100/4 (due to strides=2 in encoder)
x = layers.Reshape((25, 64))(x)
x = layers.Conv1DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv1DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_1 (Dense)             (None, 1600)              104000    
                                                                 
 reshape (Reshape)           (None, 25, 64)            0         
                                                                 
 conv1d_transpose (Conv1DTra  (None, 50, 64)           12352     
 nspose)                                                         
                                                                 
 conv1d_transpose_1 (Conv1DT  (None, 100, 32)          6176      
 ranspose)                                                       
                                                                 
 conv1d_transpose_2 (Conv1DT  (None, 100, 3)           291 

In [7]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=-1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [9]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(data, epochs=500, batch_size=128)

Epoch 1/500
138/138 [==============================] - 4s 14ms/step - loss: 61.1393 - reconstruction_loss: 61.2808 - kl_loss: 1.1210e-04
Epoch 2/500
138/138 [==============================] - 2s 14ms/step - loss: 61.2742 - reconstruction_loss: 61.2754 - kl_loss: 1.0098e-04
Epoch 3/500
138/138 [==============================] - 2s 14ms/step - loss: 61.3990 - reconstruction_loss: 61.3344 - kl_loss: 1.2986e-04
Epoch 4/500
138/138 [==============================] - 2s 14ms/step - loss: 61.3095 - reconstruction_loss: 61.3745 - kl_loss: 9.9320e-05
Epoch 5/500
138/138 [==============================] - 2s 14ms/step - loss: 61.3485 - reconstruction_loss: 61.3499 - kl_loss: 1.0139e-04
Epoch 6/500
138/138 [==============================] - 2s 15ms/step - loss: 61.2583 - reconstruction_loss: 61.2746 - kl_loss: 1.1547e-04
Epoch 7/500
138/138 [==============================] - 2s 13ms/step - loss: 61.2453 - reconstruction_loss: 61.3062 - kl_loss: 1.2647e-04
Epoch 8/500
138/138 [====================

KeyboardInterrupt: 